#### 這個Notebook參考了這篇文章
https://itnext.io/visualize-your-rag-data-eda-for-retrieval-augmented-generation-0701ee98768f
#### 上面作者的Code
https://github.com/Renumics/renumics-rag/blob/main/notebooks/visualize_rag_tutorial.ipynb
#### 其他相關文章
##### Llama: https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/tree/main<br>
##### Laungchain: https://python.langchain.com/docs/integrations/chat/llama2_chat<br>
##### Quantization: https://chih-sheng-huang821.medium.com/ai%E6%A8%A1%E5%9E%8B%E5%A3%93%E7%B8%AE%E6%8A%80%E8%A1%93-%E9%87%8F%E5%8C%96-quantization-966505128365<br>
##### Embedding: https://medium.com/@fredericklee_73485/word-embedding%E5%92%8Cword2vec%E7%B0%A1%E4%BB%8B-c9c874f48364

In [33]:
import os

os.environ["OPENAI_API_KEY"] = ''

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma

# 透過OpenAIEmbedding做Embedding
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")
docs_vectorstore = Chroma(
    collection_name="docs_store",
    embedding_function=embeddings_model,
    persist_directory="html-db"
)

In [35]:
from langchain_community.document_loaders import BSHTMLLoader, DirectoryLoader

# 解析hmtl file
loader = DirectoryLoader(
    "/Users/rich/Desktop/Advanced-ML/RAG/Demo/html",
    glob="*.html",
    loader_cls=BSHTMLLoader,
    loader_kwargs={"open_encoding": "utf-8"},
    recursive=True,
    show_progress=True,
)
docs = loader.load()

100%|██████████| 7/7 [00:01<00:00,  4.52it/s]


In [36]:
# 切成100個chunk size 太大的話token會爆掉
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=20, add_start_index=True
)
splits = text_splitter.split_documents(docs)

In [37]:
# 使用hash function 打亂數據並給定唯一的id
import hashlib
import json
from langchain.docstore.document import Document

def stable_hash(doc: Document) -> str:
    return hashlib.sha256(json.dumps(doc.metadata, sort_keys=True).encode()).hexdigest()

unique_docs = []
unique_ids = set()

for doc in splits:
    doc_id = stable_hash(doc)
    if doc_id not in unique_ids:
        unique_docs.append(doc)
        unique_ids.add(doc_id)

docs_vectorstore.add_documents(unique_docs, ids=list(unique_ids))
docs_vectorstore.persist()

In [38]:
# 創建Retriever, 選擇top k=20
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0.0)
retriever = docs_vectorstore.as_retriever(search_kwargs={"k": 20})

In [39]:
# 建立Prompt template
from langchain_core.prompts import ChatPromptTemplate

template = """
You are an assistant for question-answering tasks.
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES").
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: {question}
=========
{source_documents}
=========
FINAL ANSWER: """
prompt = ChatPromptTemplate.from_template(template)

In [40]:
from typing import List

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 轉換一下document
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(
        f"Content: {doc.page_content}\nSource: {doc.metadata['source']}" for doc in docs
    )

# 格式化ouptu
rag_chain_from_docs = (
    RunnablePassthrough.assign(
        source_documents=(lambda x: format_docs(x["source_documents"]))
    )
    | prompt
    | llm
    | StrOutputParser()
)

# 建立平行處理流程產生過ans
rag_chain = RunnableParallel(
    {
        "source_documents": retriever,
        "question": RunnablePassthrough(),
    }
).assign(answer=rag_chain_from_docs)

In [41]:
question = "日本好像最近有地震, 你知道是哪裡嗎？ 規模和強度多大, 好像有引發其他自然災害"
rag_response = rag_chain.invoke(question)
rag_answer = rag_response["answer"]
print(rag_answer)

最近的地震發生在日本的能登半島，規模為7.6級。這次地震的最大震度7在志賀町及輪島市測得，是氣象廳震度等級最強烈等級。此外，這次地震還引發了海嘯，並導致石川縣多棟建築物倒塌，至少有36,000戶家庭斷電。另外，受地震影響，石川縣珠洲市川浦町的海岸線往海洋的方向移動約175公尺，災區沿岸多出240公頃的陸地。

SOURCES: 
- 2024年能登半島地震 - 維基百科，自由的百科全書.html
- 日本石川7.6強震發「大海嘯警報」　中央氣象署也發聲了. TVBS新聞網. 2024-01-01
- 日本地震｜能登7.6級強震屬「餘震」？專家：或與地下水有關. 2024-01-02
- 日本地理學會調查隊於2024年1月4日發表能登半島海岸地形首次報告，表示受本次地震活動影響，石川県珠洲市川浦町的海岸線往海洋的方向移動約175公尺[92]。災區沿岸多出240公頃的陸地[93]。東京


In [42]:
from langchain.schema import HumanMessage, AIMessage

human_message = HumanMessage(content=question)
default_answer = llm([human_message])
print(default_answer)

content='對不起，作為一個AI，我無法提供即時的新聞更新或災害報告。請查詢最新的新聞來獲得最準確的信息。' response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 57, 'total_tokens': 117}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-ea8cc6b2-8c40-4ce3-af8a-bd6aece1532c-0'


In [43]:
# 把output轉換成df
import pandas as pd

response = docs_vectorstore.get(include=["metadatas", "documents", "embeddings"])
df = pd.DataFrame(
    {
        "id": response["ids"],
        "source": [metadata.get("source") for metadata in response["metadatas"]],
        "document": response["documents"],
        "embedding": response["embeddings"],
    }
)

df

,id,source,document,embedding
0,0032519935d77c04b3fdd6496f68aa315314f2a4e5e05d...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,"Jeux）[1]參賽國家及地區206（＋難民代表團）（預計）參賽運動員10,500人（預計）...","[-0.006430391222238541, -0.023623986169695854,..."
1,004a56cb9f5c43d6f03bd249e7e00a2a249202de139507...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,地震後工力所強震動觀測中心共收取距震央100公里範圍內實時強震儀記錄8組、震度儀記錄18組，...,"[-0.010626446455717087, -0.0015783696435391903..."
2,0050426e09232666b3f01fac9302d039204a563c314e5c...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,2011年 雲南盈江\n2012年 新疆新源\n2012年 江蘇揚州\n2012年 雲南彝良...,"[-0.0279183741658926, -0.027891242876648903, 0..."
3,008044e9a062d6f2952bcfd0f6e22680f230a530a1ae37...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,1\n\n●\n\n●\n\n●\n\n●\n\n1\n\n1\n\n\n\n5\n\n\n...,"[-0.0010993037139996886, -0.003849636996164918..."
4,009f079500e8a869a97c7b9a8d79c00ecb4f4f07805409...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,2\n\n1\n\n5\n\n3\n\n3\n\n5\n\n5\n\n6\n\n8\n\n9...,"[-0.003479189705103636, 0.0020086567383259535,..."
...,...,...,...,...
1343,ff1a0d671cccb42fc4d3225482e8776bc58879a2e9fbed...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,強烈搖晃。[7][8][9]主震發生後九分鐘，還發生了一次6.2的餘震[10]。,"[-0.010467174462974072, -0.020857101306319237,..."
1344,ff329630356ab2628c855aae443a1ae1998416efa7ae6d...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,2003年 新疆昭蘇\n2005年 江西九江\n2006年 雲南鹽津\n2008年 四川汶川...,"[-0.0169531237334013, -0.030072422698140144, 0..."
1345,ff36fefd57b10cebb7d566c16957f183a8615e504f6422...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,本頁面最後修訂於2024年3月27日 (星期三) 09:32。\n本站的全部文字在創用CC ...,"[-0.002300066174939275, -0.006432190537452698,..."
1346,ff845fe4fab0cc09835733eaf5e6f59e961b2a3ed48c68...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,facilities in the Paris area (ex. Versailles),"[-0.009839951992034912, -0.018048899248242378,..."


In [44]:
from chromadb.utils import embedding_functions

results = docs_vectorstore.similarity_search_with_score(question, k=10)

results

[(Document(page_content='^ 日本地震｜能登7.6級強震屬「餘震」？專家：或與地下水有關. 2024-01-02  [2024-01-02]. （原始內容存檔於2024-01-02）.', metadata={'source': '/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/2024年能登半島地震 - 維基百科，自由的百科全書.html', 'start_index': 26430, 'title': '2024年能登半島地震 - 維基百科，自由的百科全書'}),
  0.23312021792779186),
 (Document(page_content='逆斷層活動。根據日本氣象廳報告，本次地震最大震度7[6]在志賀町及輪島市測得，是氣象廳震度等級最強烈等級，為繼1995年阪神淡路大震災、2004年新潟縣中越地震、2011年東日本大震災、2016年熊本', metadata={'source': '/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/2024年能登半島地震 - 維基百科，自由的百科全書.html', 'start_index': 2536, 'title': '2024年能登半島地震 - 維基百科，自由的百科全書'}),
  0.23812513053417206),
 (Document(page_content='在這種作用下，日本的日本海沿岸出現了一系列斷層，並可能引發許多重大地震。例如，1833年莊內衝地震（英語：1833 Shōnai earthquake）、1940年積丹地震（英語：1940', metadata={'source': '/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/2024年能登半島地震 - 維基百科，自由的百科全書.html', 'start_index': 3108, 'title': '2024年能登半島地震 - 維基百科，自由的百科全書'}),
  0.24251015484333038),
 (Document(page_content='earthquake）、於1964年發生的M7.5的新潟地震，與1983年發生的M7.7日本海中部地震、1993

In [45]:
# 創建一個新列表來存儲相似文檔及其分數
similar_docs = []

# 遍歷返回的(Document, score)元組
for doc, score in results:
    similar_docs.append({
        'similarity_score': score
    })

# 創建一個新的DataFrame
similar_df = pd.DataFrame(similar_docs)

# 將新的DataFrame連接到原始DataFrame
df = pd.concat([df, similar_df], axis=1)

In [46]:
df.head(10)

,id,source,document,embedding,similarity_score
0,0032519935d77c04b3fdd6496f68aa315314f2a4e5e05d...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,"Jeux）[1]參賽國家及地區206（＋難民代表團）（預計）參賽運動員10,500人（預計）...","[-0.006430391222238541, -0.023623986169695854,...",0.233120
1,004a56cb9f5c43d6f03bd249e7e00a2a249202de139507...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,地震後工力所強震動觀測中心共收取距震央100公里範圍內實時強震儀記錄8組、震度儀記錄18組，...,"[-0.010626446455717087, -0.0015783696435391903...",0.238125
2,0050426e09232666b3f01fac9302d039204a563c314e5c...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,2011年 雲南盈江\n2012年 新疆新源\n2012年 江蘇揚州\n2012年 雲南彝良...,"[-0.0279183741658926, -0.027891242876648903, 0...",0.242510
3,008044e9a062d6f2952bcfd0f6e22680f230a530a1ae37...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,1\n\n●\n\n●\n\n●\n\n●\n\n1\n\n1\n\n\n\n5\n\n\n...,"[-0.0010993037139996886, -0.003849636996164918...",0.244523
4,009f079500e8a869a97c7b9a8d79c00ecb4f4f07805409...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,2\n\n1\n\n5\n\n3\n\n3\n\n5\n\n5\n\n6\n\n8\n\n9...,"[-0.003479189705103636, 0.0020086567383259535,...",0.245821
5,00b500a007220c5c91f07967fee85e09886adb6764ca71...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,1998年 河北張北（英語：1998 Zhangbei–Shangyi earthquake...,"[-0.00762561708688736, -0.030845193192362785, ...",0.246284
6,00c3e649074cc03d90e53af70dc121cf383952700e053f...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,澳洲航空30號班機事故：在巡航高度，貨艙發生輕微爆炸\n西南航空1380號班機事故：在高空爬...,"[-0.0036662116181105375, -0.01275494322180748,...",0.248442
7,00de1957adbd4f498e03681e26097e6c6f87e591ba41d2...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,^ NTV News. 地震の影響で北陸新幹線をはじめ東北、秋田、北陸新幹線運転見合わせ 午...,"[-0.014836885035037994, -0.02803885191679001, ...",0.250221
8,0136ed5570cfbda793d61c75e0a27060145941d1b0e9e9...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,"977\n\n9,368\n\n13,313\n\n55,538\n\n6\n\n19\n\...","[0.002013698685914278, -0.007452691905200481, ...",0.255528
9,0137af7f5b1e2c1a0e6a8fa0442376f68af101b9628cc0...,/Users/rich/Desktop/Advanced-ML/RAG/Demo/html/...,薩克斯坦災難2024年俄羅斯2020年代吉爾吉斯2020年代哈薩克庫爾干州隱藏分類：​引文格...,"[-0.011932240799069405, -0.007539006415754557,...",0.255772
